# Introduction
This notebook is the next step in my data exploration of this dataset. You can view the previous notebook [here](https://www.kaggle.com/code/syaugialkaf/1-e-commerce-c-t-data-exploration#Import-Necessary-Libraries).
In this step, we will add, remove, and tweak some features in the e-commerce clickstream and transaction dataset. This process follows what I believe is best for the next step (Analysis), guided by my earlier notebook linked above. Feel free to share your opinions in the comment section.

# Data Load
## Import Necessary Library

In [1]:
import numpy as np
import pandas as pd

## Load The Data

In [2]:
file_path = '/kaggle/input/e-commerce-clickstream-and-transaction-dataset/ecommerce_clickstream_transactions.csv'
df = pd.read_csv(file_path)

In [3]:
df

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN
3,1,1,2024-03-12 00:32:05.495638,add_to_cart,prod_4112,NaN,NaN
4,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,NaN
...,...,...,...,...,...,...,...
74812,1000,10,2024-05-11 22:48:45.500117,purchase,prod_1238,186.208225,purchase
74813,1000,10,2024-03-29 04:09:32.514318,logout,NaN,NaN,NaN
74814,1000,10,2024-02-09 02:58:56.128697,login,NaN,NaN,NaN
74815,1000,10,2024-04-30 16:19:48.002633,purchase,prod_2515,198.757849,purchase


# Sort The Data
In this section, we will sort the data to better understand user behavior. Since the `SessionID` was not highly relevant in analyzing the clickstream sequence, we will drop it from the table. The data will be sorted based on `Timestamp` and `UserID`. The purpose of this sorting is to reveal the sequence of user events. This could help us discover patterns that lead to a user purchasing a product or allow us to analyze trends over time. This is an exciting step that can provide valuable insights!

Below is the code to convert the `Timestamp` to a datetime object, sort the DataFrame by `UserID` and `Timestamp`, and drop the `SessionID` column:

In [4]:
# Convert 'Timestamp' to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
# Sort the df_n by 'UserID' and 'Timestamp'
df_sorted = df.sort_values(by=['UserID', 'Timestamp'])
df_sorted = df_sorted.drop(columns=['SessionID'])
df_sorted

,UserID,Timestamp,EventType,ProductID,Amount,Outcome
5,1,2024-01-01 23:09:51.956825,page_view,NaN,NaN,NaN
32,1,2024-01-02 00:15:51.420238,add_to_cart,prod_6144,NaN,NaN
36,1,2024-01-03 23:51:05.729189,logout,NaN,NaN,NaN
29,1,2024-01-04 09:25:49.705624,add_to_cart,prod_1102,NaN,NaN
65,1,2024-01-04 17:09:29.677060,click,NaN,NaN,NaN
...,...,...,...,...,...,...
74752,1000,2024-07-10 09:34:52.379949,product_view,prod_7884,NaN,NaN
74748,1000,2024-07-11 12:40:44.578010,logout,NaN,NaN,NaN
74810,1000,2024-07-13 15:30:59.991899,add_to_cart,prod_1899,NaN,NaN
74794,1000,2024-07-15 08:49:32.723612,click,NaN,NaN,NaN


# Handling Missing Values
In this section, we will address the missing values (NaNs) in our dataset to ensure the data is clean and ready for analysis. We will replace NaN values in the `ProductID`, `Amount`, and `Outcome` columns with appropriate default values. This will help prevent issues during analysis and modeling.

**Steps to Handle NaN Values:**
- Replace NaNs in `ProductID` with `Unknown`: This provides a clear indication that the product ID is not available.
- Replace NaNs in `Amount` with `0`: This assumes that missing amounts indicate no transaction amount.
- Replace NaNs in `Outcome` with `0`: This assumes that missing outcomes indicate no significant outcome (e.g., no purchase).

Additionally, we will convert any 'purchase' values in the Outcome column to 1, to simplify binary outcome analysis.

In [5]:
# Replacing NaN values
df_sorted['ProductID'].fillna('Unknown', inplace=True)
df_sorted['Amount'].fillna(0, inplace=True)
df_sorted['Outcome'].fillna(0, inplace=True)

# Replacing 'purchase' values in Outcome with 1
df_sorted['Outcome'] = df_sorted['Outcome'].apply(lambda x: 1 if x == 'purchase' else x)

/tmp/ipykernel_18/3582132557.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_sorted['ProductID'].fillna('Unknown', inplace=True)
/tmp/ipykernel_18/3582132557.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

In [6]:
df_sorted.head(10)

,UserID,Timestamp,EventType,ProductID,Amount,Outcome
5,1,2024-01-01 23:09:51.956825,page_view,Unknown,0.000000,0
32,1,2024-01-02 00:15:51.420238,add_to_cart,prod_6144,0.000000,0
36,1,2024-01-03 23:51:05.729189,logout,Unknown,0.000000,0
29,1,2024-01-04 09:25:49.705624,add_to_cart,prod_1102,0.000000,0
65,1,2024-01-04 17:09:29.677060,click,Unknown,0.000000,0
72,1,2024-01-06 04:33:39.275154,page_view,Unknown,0.000000,0
62,1,2024-01-13 19:09:28.166082,product_view,prod_9816,0.000000,0
51,1,2024-01-17 08:27:34.705063,add_to_cart,prod_5915,0.000000,0
21,1,2024-01-19 15:04:33.065650,page_view,Unknown,0.000000,0
34,1,2024-01-19 16:51:29.413903,purchase,prod_6690,149.270047,1


# Adding More Features
In this section, we will add several new features to our dataset to enhance our analysis capabilities. These features include `ProductCount`, `EventCount`, `DaysSinceLastPurchase`, and indicators for events preceding a purchase. We will also split the `Timestamp` into separate components such as hour, day, month, and year. 

To begin, we will create a backup of our sorted DataFrame.

## Step 1: Create a Backup of the DataFrame
First, let's create a new DataFrame df_n as a backup of our sorted DataFrame.

In [7]:
df_n = df_sorted

## Step 2: Adding Product Count
We will count the number of products a user interacts with before making a purchase. This feature can help us understand user behavior better.

In [8]:
df_n['Timestamp'] = pd.to_datetime(df_n['Timestamp'])
df_n = df_n.sort_values(by=['UserID', 'Timestamp'])

# Initialize the ProductCount column
df_n['ProductCount'] = 0

checkpoint = 0

# Iterate over rows to count products before a purchase
for user_id in df_n['UserID'].unique():
    user_df_n = df_n[df_n['UserID'] == user_id]
    product_count = 0
    for idx in range(len(user_df_n)):
        row = user_df_n.iloc[idx]
        if row['Outcome'] == 1:
            checkpoint = 1
        elif checkpoint != 0:
            product_count = 0
            checkpoint = 0
        elif row['ProductID'] != 'Unknown':
            product_count += 1
        df_n.loc[row.name, 'ProductCount'] = product_count


## Step 3: Adding Event Count
Next, we will count the number of events a user performs before making a purchase. This feature helps in analyzing user engagement.

In [9]:
# Initialize the ProductCount column
df_n['EventCount'] = 0

checkpoint = 0

# Iterate over rows to count products before a purchase
for user_id in df_n['UserID'].unique():
    user_df_n = df_n[df_n['UserID'] == user_id]
    event_count = 0
    for idx in range(len(user_df_n)):
        row = user_df_n.iloc[idx]
        if row['Outcome'] == 1:
            checkpoint = 1
        elif checkpoint != 0:
            event_count = 0
            checkpoint = 0
        elif row['Outcome'] != 1:
            event_count += 1
        df_n.loc[row.name, 'EventCount'] = event_count

## Step 4: Adding Days Since Last Purchase
We will calculate the number of days since the user's last purchase. This feature can be useful for time-based analysis.

In [10]:
df_n['DaysSinceLastPurchase'] = None  # New column for days since the last purchase

row = df_n.iloc[0]
first_date = row['Timestamp'].date()

for user_id in df_n['UserID'].unique():
    user_df_n = df_n[df_n['UserID'] == user_id]
    last_purchase_date = first_date
    last_purchase_days = None  # Variable to store days since last purchase for current row
    
    for idx in range(len(user_df_n)):
        row = user_df_n.iloc[idx]
        current_date = row['Timestamp'].date()
        
        if row['Outcome'] == 1:  # Update last purchase date on purchase
            if last_purchase_date is not None:
                last_purchase_days = (current_date - last_purchase_date).days
            last_purchase_date = current_date
            days_since_last_event = last_purchase_days if last_purchase_days is not None else 0
            last_purchase_days = None  # Reset last_purchase_days after purchase
        else:
            if last_purchase_date is not None:
                days_since_last_event = (current_date - last_purchase_date).days
            else:
                days_since_last_event = 0  # or NaN if you prefer
        
        df_n.loc[row.name, 'DaysSinceLastPurchase'] = days_since_last_event

## Step 5: Adding Purchase Indicators
We will add indicators for events that occurred immediately before a purchase. This can help in understanding the sequence of user actions leading to a purchase.

In [11]:
# Initialize new columns
df_n['OneEventBeforePurchase'] = 0
df_n['TwoEventBeforePurchase'] = 0
df_n['ThreeEventBeforePurchase'] = 0

# Iterate over each user
for user_id in df_n['UserID'].unique():
    user_df_n = df_n[df_n['UserID'] == user_id]
    
    # Create a list to store the last three events before each purchase
    event_buffer = []

    # Iterate over rows to process events
    for idx in range(len(user_df_n)):
        row = user_df_n.iloc[idx]

        if row['Outcome'] == 1:  # If the event is a purchase
            # Mark the events before this purchase
            if len(event_buffer) > 0:
                if len(event_buffer) >= 1:
                    df_n.loc[event_buffer[-1], 'OneEventBeforePurchase'] = 1
                if len(event_buffer) >= 2:
                    df_n.loc[event_buffer[-2], 'TwoEventBeforePurchase'] = 1
                if len(event_buffer) >= 3:
                    df_n.loc[event_buffer[-3], 'ThreeEventBeforePurchase'] = 1
            # Clear the event buffer after processing a purchase
            event_buffer = []
        else:
            # Add current event index to the buffer
            event_buffer.append(row.name)
            # Keep only the last three events in the buffer
            if len(event_buffer) > 3:
                event_buffer.pop(0)

## Step 6: Splitting the Timestamp
Finally, we will split the Timestamp into separate components such as hour, day, month, and year for more granular time-based analysis.

In [12]:
# Extract Hour, Day, Month, and Year
df_n['Hour'] = df_n['Timestamp'].dt.hour
df_n['Day'] = df_n['Timestamp'].dt.day
df_n['Month'] = df_n['Timestamp'].dt.month
df_n['Year'] = df_n['Timestamp'].dt.year

In [13]:
df_n.head()

,UserID,Timestamp,EventType,ProductID,Amount,Outcome,ProductCount,EventCount,DaysSinceLastPurchase,OneEventBeforePurchase,TwoEventBeforePurchase,ThreeEventBeforePurchase,Hour,Day,Month,Year
5,1,2024-01-01 23:09:51.956825,page_view,Unknown,0.0,0,0,1,0,0,0,0,23,1,1,2024
32,1,2024-01-02 00:15:51.420238,add_to_cart,prod_6144,0.0,0,1,2,1,0,0,0,0,2,1,2024
36,1,2024-01-03 23:51:05.729189,logout,Unknown,0.0,0,1,3,2,0,0,0,23,3,1,2024
29,1,2024-01-04 09:25:49.705624,add_to_cart,prod_1102,0.0,0,2,4,3,0,0,0,9,4,1,2024
65,1,2024-01-04 17:09:29.677060,click,Unknown,0.0,0,2,5,3,0,0,0,17,4,1,2024


# Feature Re-order
In this step, we will reorganize the columns of our DataFrame to ensure a logical and intuitive order. This will make our dataset easier to understand and work with. After reordering the columns, we will save the enhanced DataFrame to a CSV file.

In [14]:
df_final = df_n

# Define the new column order
new_column_order = [
    'UserID', 'Timestamp', 'Year', 'Month', 'Day', 'Hour', 
    'EventType', 'ProductID', 'Amount', 'Outcome', 
    'ProductCount', 'EventCount', 'DaysSinceLastPurchase', 
    'OneEventBeforePurchase', 'TwoEventBeforePurchase', 
    'ThreeEventBeforePurchase'
]

# Reorder the columns
df_final = df_final[new_column_order]
df_final

,UserID,Timestamp,Year,Month,Day,Hour,EventType,ProductID,Amount,Outcome,ProductCount,EventCount,DaysSinceLastPurchase,OneEventBeforePurchase,TwoEventBeforePurchase,ThreeEventBeforePurchase
5,1,2024-01-01 23:09:51.956825,2024,1,1,23,page_view,Unknown,0.0,0,0,1,0,0,0,0
32,1,2024-01-02 00:15:51.420238,2024,1,2,0,add_to_cart,prod_6144,0.0,0,1,2,1,0,0,0
36,1,2024-01-03 23:51:05.729189,2024,1,3,23,logout,Unknown,0.0,0,1,3,2,0,0,0
29,1,2024-01-04 09:25:49.705624,2024,1,4,9,add_to_cart,prod_1102,0.0,0,2,4,3,0,0,0
65,1,2024-01-04 17:09:29.677060,2024,1,4,17,click,Unknown,0.0,0,2,5,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74752,1000,2024-07-10 09:34:52.379949,2024,7,10,9,product_view,prod_7884,0.0,0,6,16,52,0,0,0
74748,1000,2024-07-11 12:40:44.578010,2024,7,11,12,logout,Unknown,0.0,0,6,17,53,0,0,0
74810,1000,2024-07-13 15:30:59.991899,2024,7,13,15,add_to_cart,prod_1899,0.0,0,7,18,55,0,0,0
74794,1000,2024-07-15 08:49:32.723612,2024,7,15,8,click,Unknown,0.0,0,7,19,57,0,0,0


# Save the Enhanced DataFrame to CSV
Finally, we will save the reordered DataFrame to a CSV file for future analysis.

By reordering the columns and saving the enhanced dataset, we ensure that our data is well-organized and ready for further analysis.

In [15]:
df_final.to_csv('ecommerce_clickstream_transactions_enhanced.csv', index=False)